In [22]:
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

# Anomaly Detection

In [23]:
df = pd.read_json('anomaly_detection/results/aucs.json')
df = df.drop(index='catch')
df = df.round(3)

ranks = df.rank(axis=0, method='min', ascending=False)
mean_ranks = ranks.mean(axis=1)

df_show = df.copy()
df['mean'] = df.mean(axis=1)
df = df.round(3)
df['mean_rank'] = mean_ranks
df.sort_values('mean_rank', inplace=True)
df

,nyc_taxi,ec2_request_latency_system_failure,msl,swat,smap,smd,mean,mean_rank
patchtrad,0.922,0.999,0.661,0.845,0.660,0.869,0.826,2.333333
patchtst,0.696,0.999,0.626,0.843,0.622,0.882,0.778,4.333333
jepatchtrad_base,0.848,0.994,0.630,0.815,0.689,0.859,0.806,5.500000
patchad,0.972,0.998,0.622,0.822,0.671,0.818,0.817,5.833333
gat,0.689,0.999,0.617,0.816,0.646,0.820,0.764,6.666667
jepatchtrad_small,0.672,0.995,0.631,0.815,0.645,0.847,0.768,7.000000
aelstm,0.664,0.998,0.612,0.840,0.618,0.828,0.760,8.333333
lstm,0.511,0.999,0.595,0.842,0.604,0.833,0.731,8.333333
tranad,0.551,0.967,0.644,0.815,0.581,0.884,0.740,8.500000
lstm_rev,0.646,0.998,0.627,0.238,0.586,0.858,0.659,9.500000


# Forecasting

## Small

In [24]:
df = pd.read_json('forecasting/results/mse_small.json')

ranks = df.rank(axis=0, method='min', ascending=True)
mean_ranks = ranks.mean(axis=1)
df['mean_rank'] = mean_ranks
df.sort_values('mean_rank', inplace=True)

df["encoder"]=" "
df["scratch"]=" "

for i in range(len(df)):
    index_name = df.index[i]
    index_name_split = index_name.split("_")
    
    if index_name_split[-1]=="revin":
        index_name_split = [index_name_split[0], index_name_split[-1], index_name_split[1], index_name_split[2]]

    if index_name_split[-1]=="True":
        df.at[index_name, "scratch"] = "True"
        df.at[index_name, "encoder"] = "trainable"
        df.index.values[i] = "PatchTST"
        if index_name_split[1]=="revin":
            df.index.values[i] += "_Revin"
    else:
        if index_name_split[-2]=="True":
            df.at[index_name, "scratch"] = "False"
            df.at[index_name, "encoder"] = "freeze"
        else:
            df.at[index_name, "scratch"] = "False"
            df.at[index_name, "encoder"] = "trainable"
        df.index.values[i] = "JEPAtchTST"
        if index_name_split[1]=="revin":
            df.index.values[i] += "_Revin"
df.T

,JEPAtchTST_Revin,PatchTST_Revin,JEPAtchTST_Revin,JEPAtchTST,PatchTST,JEPAtchTST
etth1,0.466096,0.500651,0.521106,0.461953,0.512409,0.505454
etth2,0.173743,0.170718,0.178021,0.182287,0.201699,0.21385
ettm1_univariate,0.041811,0.041256,0.042465,0.052844,0.048413,0.091453
ettm2_univariate,0.122536,0.13115,0.145015,0.130941,0.143944,0.23163
ettm1,0.559685,0.626324,0.653843,0.499268,0.536895,0.542597
ettm2,0.124991,0.125961,0.131073,0.129927,0.137828,0.173341
etth1_univariate,0.102284,0.101346,0.103073,0.184523,0.104248,0.191407
etth2_univariate,0.258618,0.260639,0.271762,0.304793,0.356557,0.347272
exchange_rate,0.058823,0.056895,0.06005,0.15336,0.08904,0.179716
exchange_rate_univariate,0.074473,0.073896,0.072805,0.080387,0.081563,0.138697


## Base

In [25]:
df = pd.read_json('forecasting/results/mse_base.json')
ranks = df.rank(axis=0, method='min', ascending=True)
mean_ranks = ranks.mean(axis=1)
df['mean_rank'] = mean_ranks
df.sort_values('mean_rank', inplace=True)

df["encoder"]=" "
df["scratch"]=" "

for i in range(len(df)):
    index_name = df.index[i]
    index_name_split = index_name.split("_")
    
    if index_name_split[-1]=="revin":
        index_name_split = [index_name_split[0], index_name_split[-1], index_name_split[1], index_name_split[2]]

    if index_name_split[-1]=="True":
        df.at[index_name, "scratch"] = "True"
        df.at[index_name, "encoder"] = "trainable"
        df.index.values[i] = "PatchTST"
        if index_name_split[1]=="revin":
            df.index.values[i] += "_Revin"
    else:
        if index_name_split[-2]=="True":
            df.at[index_name, "scratch"] = "False"
            df.at[index_name, "encoder"] = "freeze"
        else:
            df.at[index_name, "scratch"] = "False"
            df.at[index_name, "encoder"] = "trainable"
        df.index.values[i] = "JEPAtchTST"
        if index_name_split[1]=="revin":
            df.index.values[i] += "_Revin"
df.T

df.T

,JEPAtchTST_Revin,PatchTST,PatchTST_Revin,JEPAtchTST,JEPAtchTST_Revin,JEPAtchTST
etth1,0.45744,0.460798,0.48934,0.458688,0.460496,0.482329
etth2,0.173638,0.209032,0.178488,0.19144,0.183851,0.205626
ettm1,0.453019,0.444751,0.422134,0.445688,0.45693,0.47631
ettm2,0.118655,0.115601,0.121659,0.145122,NaN,0.130069
weather,NaN,0.186497,NaN,0.245535,NaN,0.206712
etth1_univariate,0.099272,0.106821,0.100541,0.158228,0.118087,0.218047
etth2_univariate,0.228098,0.283865,0.261876,0.274855,0.259427,0.278312
ettm1_univariate,0.042571,0.054934,0.04434,0.0999,0.046131,0.076638
ettm2_univariate,NaN,0.13998,NaN,0.2059,NaN,0.140886
national_illness_univariate,NaN,4.181062,NaN,3.112235,NaN,3.825607


## Big

In [26]:
df = pd.read_json('forecasting/results/mse_big.json')

ranks = df.rank(axis=0, method='min', ascending=True)
mean_ranks = ranks.mean(axis=1)
df['mean_rank'] = mean_ranks
df.sort_values('mean_rank', inplace=True)

df["encoder"]=" "
df["scratch"]=" "

for i in range(len(df)):
    index_name = df.index[i]
    index_name_split = index_name.split("_")
    
    if index_name_split[-1]=="revin":
        index_name_split = [index_name_split[0], index_name_split[-1], index_name_split[1], index_name_split[2]]

    if index_name_split[-1]=="True":
        df.at[index_name, "scratch"] = "True"
        df.at[index_name, "encoder"] = "trainable"
        df.index.values[i] = "PatchTST"
        if index_name_split[1]=="revin":
            df.index.values[i] += "_Revin"
    else:
        if index_name_split[-2]=="True":
            df.at[index_name, "scratch"] = "False"
            df.at[index_name, "encoder"] = "freeze"
        else:
            df.at[index_name, "scratch"] = "False"
            df.at[index_name, "encoder"] = "trainable"
        df.index.values[i] = "JEPAtchTST"
        if index_name_split[1]=="revin":
            df.index.values[i] += "_Revin"
df.T
df.T

,PatchTST_Revin,PatchTST,JEPAtchTST_Revin,JEPAtchTST_Revin,JEPAtchTST,JEPAtchTST
etth1_univariate,0.094088,0.107451,0.122483,0.11747,0.166651,0.470768
etth1,0.449929,0.447328,0.469782,1.234548,0.457347,1.177813
etth2,0.170635,NaN,0.195744,0.23248,NaN,NaN
etth2_univariate,0.217244,NaN,0.335347,0.338635,NaN,NaN
ettm1,0.417132,NaN,NaN,0.951198,NaN,NaN
mean_rank,1.2,1.5,3.0,3.4,4.0,5.5
encoder,trainable,trainable,trainable,freeze,trainable,freeze
scratch,True,True,False,False,False,False


# Classification

### Artificial Signals

In [49]:
small = pd.read_json('classification/type1/results/small/accs.json')
base = pd.read_json('classification/type1/results/base/accs.json')
big = pd.read_json('classification/type1/results/big/accs.json')

small.columns = ['small']
base.columns = ['base']
big.columns = ['big']

name_map = {
    "JePatchTST_True_True": "scratch",
    "JePatchTST_True_False": "freeze",
    "JePatchTST_False_False": "finetune"
}

df = pd.concat([small, base, big], axis=1)
df = df.rename(index=name_map)
df = df.round(3)
df.T

,scratch,freeze,finetune
small,0.547,0.713,0.747
base,0.700,0.767,0.900
big,0.653,0.747,0.800


## True Signals

In [50]:
small = pd.read_json('classification/type2/results/small/accs.json')
base = pd.read_json('classification/type2/results/base/accs.json')
big = pd.read_json('classification/type2/results/big/accs.json')

small.columns = ['small']
base.columns = ['base']
big.columns = ['big']

name_map = {
    "JePatchTST_True_True": "scratch",
    "JePatchTST_True_False": "freeze",
    "JePatchTST_False_False": "finetune"
}

df = pd.concat([small, base, big], axis=1)
df = df.rename(index=name_map)
df = df.round(3)
df.T

,scratch,freeze,finetune
small,0.441,0.594,0.809
base,0.561,0.820,0.915
big,0.631,0.728,0.889
